In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import requests

BASE_URL = "https://kosis.kr/openapi/Param/statisticsParameterData.do"

params = {
    "method": "getList",
    "apiKey": "ZWRiNzEyMzEwNGI5OWQ3NzcxNGM1MDNiOGJkOTQ0Y2M=",
    "itmId": "T2+",
    "objL1": "3404+3408+3436+3531+3642+3704+",
    "objL2": "S0+",
    "objL3": "",
    "objL4": "",
    "objL5": "",
    "objL6": "",
    "objL7": "",
    "objL8": "",
    "format": "json",
    "jsonVD": "Y",
    "prdSe": "H",
    "newEstPrdCnt": "7",
    "orgId": "101",
    "tblId": "DT_1ES3A02S",
}

response = requests.get(BASE_URL, params=params)
df_raw = pd.DataFrame(response.json())

In [ ]:
keep_cols = ["PRD_DE", "C1", "C1_NM", "DT"]
missing = [c for c in keep_cols if c not in df_raw.columns]
if missing:
    raise ValueError(f"employment_raw에 필요한 컬럼이 없습니다: {missing}")

emp = df_raw[keep_cols].copy()
emp.columns = ["기간", "행정코드", "지역명", "값"]

emp["행정코드"] = emp["행정코드"].astype(str)
# emp["기간"] = emp["기간"].astype(str)
emp["값"] = pd.to_numeric(emp["값"], errors="coerce")

# 2) 기간 포맷 정리
# emp["기간"] = (
#     emp["기간"]
#     .astype(str)
#     .str.strip()
#     .str[:4]
#     + "-"
#     + emp["기간"].astype(str).str[4:]
# )


pivot_df = emp.pivot_table(
    index=["행정코드", "지역명"],
    columns="기간",
    values="값",
    aggfunc="mean",  # 동일 키 중복 시 평균(보통 중복 없어야 정상)
).reset_index()

pivot_df


In [ ]:
# 기간 컬럼 자동 추출 (숫자형 YYYYMM)
# pivot_df = pivot_df.loc[~(pivot_df['지역명'] == '아산시')].copy()

time_cols = sorted([c for c in pivot_df.columns if str(c).isdigit()])

plt.figure(figsize=(11, 6))

for _, row in pivot_df.iterrows():
    y = row[time_cols].astype(float).values
    plt.plot(time_cols, y, marker="o", label=row["지역명"])

plt.xlabel("Time (YYYYMM)")
plt.ylabel("Youth Employed Persons (thousand)")
plt.title("청년 경제 활동 TOP 5 (15–29)")
plt.xticks(time_cols, rotation=45)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
KEY_REGION = "완주군"
KEY_COLOR = "#FF5501"
BASE_COLOR = "#F8F8F8"

fig, ax = plt.subplots(figsize=(11, 6), facecolor="black")
ax.set_facecolor("black")

for _, row in pivot_df.iterrows():
    y = row[time_cols].astype(float).values

    region = row["지역명"]
    color = KEY_COLOR if region == KEY_REGION else BASE_COLOR
    lw = 3 if region == KEY_REGION else 1.5
    alpha = 1.0 if region == KEY_REGION else 0.6

    ax.plot(
        time_cols,
        y,
        marker="o",
        label=region,
        color=color,
        linewidth=lw,
        alpha=alpha,
    )

# ===== 텍스트 =====
ax.set_xlabel("Time (YYYYMM)", color="white")
ax.set_ylabel("청년 고용율 (천명단위)", color="white")
ax.set_title("청년 경제 활동 TOP 5 (15–29)", color="white")


# ===== 축 =====
ax.tick_params(colors="white")
for spine in ax.spines.values():
    spine.set_color("white")

# ===== legend =====
leg = ax.legend(
    facecolor="black",  # 배경
    edgecolor="white",  # 테두리
    framealpha=1,  # 투명도 (1 = 완전 불투명)
)
# 범례 텍스트 흰색
for text in leg.get_texts():
    text.set_color("white")

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import requests

BASE_URL = "https://kosis.kr/openapi/Param/statisticsParameterData.do"

params = {
    "method": "getList",
    "apiKey": "ZWRiNzEyMzEwNGI5OWQ3NzcxNGM1MDNiOGJkOTQ0Y2M=",
    "itmId": "T2+",
    "objL1": "3404+3408+3436+3531+3642+3704+",
    "objL2": "S0+",
    "objL3": "",
    "objL4": "",
    "objL5": "",
    "objL6": "",
    "objL7": "",
    "objL8": "",
    "format": "json",
    "jsonVD": "Y",
    "prdSe": "H",
    "newEstPrdCnt": "7",
    "orgId": "101",
    "tblId": "DT_1ES3A02S",
}

response = requests.get(BASE_URL, params=params)
df_raw = pd.DataFrame(response.json())

In [ ]:
# 데이터 불러오기
import re

import pandas as pd

pd.set_option("display.max_rows", 10000)

df = pd.read_csv("../../data/202401_202401_연령별인구현황_월간.csv", encoding="cp949")

df["지역명"] = df["행정구역"].str.split("(").str[0].str.strip()
df["행정코드"] = df["행정구역"].str.extract(r"\((\d+)\)").astype("Int64")

# 3. 시/군 단위만 추출
# - 시도: code % 100_000_000 == 0
# - 시군구: code % 100_000_000 != 0 AND code % 100_000 == 0
sigungu = df[
    (df["행정코드"] % 100_000_000 != 0) & (df["행정코드"] % 100_000 == 0)
].copy()

# 4. 숫자 컬럼 정리
num_cols = [
    "2024년01월_계_20~24세",
    "2024년01월_계_25~29세",
    "2024년01월_계_15~19세",
]

for c in num_cols:
    sigungu[c] = sigungu[c].astype(str).str.replace(",", "", regex=False).astype(int)

# 5. 청년(20~34) 인구 계산
sigungu["청년(20~34)인구"] = (
    sigungu["2024년01월_계_20~24세"]
    + sigungu["2024년01월_계_25~29세"]
    + sigungu["2024년01월_계_15~19세"]
)

# 6. 검색할 지역 정의
include_regions = ["아산시", "무안군", "안동시", "완주군", "홍성군", "당진시"]

sigungu["상위지역"] = sigungu["지역명"].str.split().str[0]
pattern = "|".join(map(re.escape, include_regions))
local_sigungu = sigungu[sigungu["지역명"].str.contains(pattern)].copy()

# 7. 결과 테이블 구성
result = local_sigungu[["지역명", "청년(20~34)인구"]]

all_result = pd.DataFrame()
all_result["지역명"] = local_sigungu["지역명"].copy()
all_result[202401] = local_sigungu["청년(20~34)인구"].copy()

result

In [ ]:
# 데이터 불러오기
import re

import pandas as pd

pd.set_option("display.max_rows", 10000)

df = pd.read_csv("../../data/202301_202301_연령별인구현황_월간.csv", encoding="cp949")

df["지역명"] = df["행정구역"].str.split("(").str[0].str.strip()
df["행정코드"] = df["행정구역"].str.extract(r"\((\d+)\)").astype("Int64")

# 3. 시/군 단위만 추출
# - 시도: code % 100_000_000 == 0
# - 시군구: code % 100_000_000 != 0 AND code % 100_000 == 0
sigungu = df[
    (df["행정코드"] % 100_000_000 != 0) & (df["행정코드"] % 100_000 == 0)
].copy()

# 4. 숫자 컬럼 정리
num_cols = [
    "2023년01월_계_20~24세",
    "2023년01월_계_25~29세",
    "2023년01월_계_15~19세",
]

for c in num_cols:
    sigungu[c] = sigungu[c].astype(str).str.replace(",", "", regex=False).astype(int)

# 5. 청년(20~34) 인구 계산
sigungu["청년(20~34)인구"] = (
    sigungu["2023년01월_계_20~24세"]
    + sigungu["2023년01월_계_25~29세"]
    + sigungu["2023년01월_계_15~19세"]
)

# 6. 검색할 지역 정의
include_regions = ["아산시", "무안군", "안동시", "완주군", "홍성군", "당진시"]

sigungu["상위지역"] = sigungu["지역명"].str.split().str[0]
pattern = "|".join(map(re.escape, include_regions))
local_sigungu = sigungu[sigungu["지역명"].str.contains(pattern)].copy()

# 7. 결과 테이블 구성
result = local_sigungu[["지역명", "청년(20~34)인구"]]

all_result[202301] = local_sigungu["청년(20~34)인구"].tolist().copy()

In [ ]:
# 데이터 불러오기
import re

import pandas as pd

pd.set_option("display.max_rows", 10000)

df = pd.read_csv("../../data/202201_202201_연령별인구현황_월간.csv", encoding="cp949")

df["지역명"] = df["행정구역"].str.split("(").str[0].str.strip()
df["행정코드"] = df["행정구역"].str.extract(r"\((\d+)\)").astype("Int64")

# 3. 시/군 단위만 추출
# - 시도: code % 100_000_000 == 0
# - 시군구: code % 100_000_000 != 0 AND code % 100_000 == 0
sigungu = df[
    (df["행정코드"] % 100_000_000 != 0) & (df["행정코드"] % 100_000 == 0)
].copy()

# 4. 숫자 컬럼 정리
num_cols = [
    "2022년01월_계_20~24세",
    "2022년01월_계_25~29세",
    "2022년01월_계_15~19세",
]

for c in num_cols:
    sigungu[c] = sigungu[c].astype(str).str.replace(",", "", regex=False).astype(int)

# 5. 청년(20~34) 인구 계산
sigungu["청년(20~34)인구"] = (
    sigungu["2022년01월_계_20~24세"]
    + sigungu["2022년01월_계_25~29세"]
    + sigungu["2022년01월_계_15~19세"]
)

# 6. 검색할 지역 정의
include_regions = ["아산시", "무안군", "안동시", "완주군", "홍성군", "당진시"]

sigungu["상위지역"] = sigungu["지역명"].str.split().str[0]
pattern = "|".join(map(re.escape, include_regions))
local_sigungu = sigungu[sigungu["지역명"].str.contains(pattern)].copy()

# 7. 결과 테이블 구성
result = local_sigungu[["지역명", "청년(20~34)인구"]]

all_result[202201] = local_sigungu["청년(20~34)인구"].tolist().copy()

all_result

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import requests

BASE_URL = "https://kosis.kr/openapi/Param/statisticsParameterData.do"

params = {
    "method": "getList",
    "apiKey": "ZWRiNzEyMzEwNGI5OWQ3NzcxNGM1MDNiOGJkOTQ0Y2M=",
    "itmId": "T21+",
    "objL1": "3404+3408+3436+3531+3642+3704+",
    "objL2": "",
    "objL3": "",
    "objL4": "",
    "objL5": "",
    "objL6": "",
    "objL7": "",
    "objL8": "",
    "format": "json",
    "jsonVD": "Y",
    "prdSe": "H",
    "newEstPrdCnt": "7",
    "orgId": "101",
    "tblId": "DT_1ES3A08S",
}

response = requests.get(BASE_URL, params=params)
df_raw = pd.DataFrame(response.json())

df_raw

In [ ]:
keep_cols = ["PRD_DE", "C1", "C1_NM", "DT"]
missing = [c for c in keep_cols if c not in df_raw.columns]
if missing:
    raise ValueError(f"employment_raw에 필요한 컬럼이 없습니다: {missing}")

emp = df_raw[keep_cols].copy()
emp.columns = ["기간", "행정코드", "지역명", "값"]

emp["행정코드"] = emp["행정코드"].astype(str)
# emp["기간"] = emp["기간"].astype(str)
emp["값"] = pd.to_numeric(emp["값"], errors="coerce")

# 2) 기간 포맷 정리
# emp["기간"] = (
#     emp["기간"]
#     .astype(str)
#     .str.strip()
#     .str[:4]
#     + "-"
#     + emp["기간"].astype(str).str[4:]
# )


pivot_df = emp.pivot_table(
    index=["행정코드", "지역명"],
    columns="기간",
    values="값",
    aggfunc="mean",  # 동일 키 중복 시 평균(보통 중복 없어야 정상)
).reset_index()

jan_cols = [c for c in pivot_df.columns if str(c).endswith("01")]

pivot_df = pivot_df[["행정코드", "지역명"] + jan_cols]

display(all_result)
display(pivot_df)


In [ ]:
# 1) all_result에 "시군구" 컬럼 만들기 (마지막 단어)
all_tmp = all_result.copy()
all_tmp["시군구"] = all_tmp["지역명"].astype(str).str.split().str[-1]

# 2) pivot_df도 키 컬럼 통일
pv_tmp = pivot_df.copy()
pv_tmp["시군구"] = pv_tmp["지역명"].astype(str)

# 3) 공통 기간(컬럼)만 뽑기
period_cols = [202201, 202301, 202401]

pv_tmp = pv_tmp.rename(columns=lambda c: int(c) if str(c).isdigit() else c)

# # 4) pivot_df 숫자만 *1000
pv_tmp[period_cols] = pv_tmp[period_cols].mul(1000)


display(all_tmp)
display(pv_tmp)

# # 5) 시군구 키로 merge 후 차이 계산 (all - pivot*1000)
merged = all_tmp[["시군구"] + period_cols].merge(
    pv_tmp[["시군구"] + period_cols], on="시군구", how="inner", suffixes=("_all", "_pv")
)

diff = merged[["시군구"]].copy()
for c in period_cols:
    diff[c] = merged[f"{c}_all"] - merged[f"{c}_pv"]

diff


In [ ]:
import matplotlib.pyplot as plt

# 기간 컬럼 자동 추출 (숫자형 YYYYMM)
pivot_df = pivot_df.loc[~(pivot_df["지역명"] == "아산시")].copy()

time_cols = sorted([c for c in diff.columns if str(c).isdigit()])

plt.figure(figsize=(11, 6))

for _, row in diff.iterrows():
    y = row[time_cols].astype(float).values
    plt.plot(time_cols, y, marker="o", label=row["시군구"])

plt.xlabel("Time (YYYYMM)")
plt.ylabel("Youth Employed Persons (thousand)")
plt.title("Top 5 Regions: Youth Employment Trend (15–29)")
plt.xticks(time_cols, rotation=45)
plt.legend()
plt.tight_layout()
plt.show()
